# Tokenizadores y capas de embeddings
Este notebook tiene los siguientes temarios que se ha aprendido en la teoria:
1. **Motivación**: texto discreto → números
2. **Tokenización**: $x \to (t_1,\dots,t_T)$
3. **Embeddings**: $t_i \to e_i \in \mathbb{R}^d$
4. **Subword tokenization**: BPE, WordPiece, Unigram LM, Byte-level BPE
5. **Tokens especiales** y **padding**
6. **Embeddings posicionales** (intuición)

Incluye **ejercicios intercalados**.

In [ ]:
import math
import random
from dataclasses import dataclass
from typing import List, Dict, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

## 0) Reproducibilidad (semillas)
En ML es normal que los resultados cambien por inicializaciones aleatorias.
Fijar semilla hace los experimentos repetibles.

In [ ]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed = 42
set_seed(seed)
print('Seed set with value', seed)

Seed set with value 42


## 1) Motivación: texto discreto vs redes neuronales
Un texto es una secuencia de símbolos discretos (caracteres/palabras). Las redes trabajan con **tensores numéricos**.

Formalmente:
$$
x \xrightarrow{\mathcal{T}} (t_1,\dots,t_T) \xrightarrow{\mathcal{E}} (e_1,\dots,e_T),\quad e_i\in\mathbb{R}^d.
$$

- $\mathcal{T}$: tokenizador (determinista) que mapea texto → tokens/IDs
- $\mathcal{E}$: embeddings (matriz entrenable) que mapea IDs → vectores

## 2) Tokenizador más simple: whitespace + vocab
Vamos a construir un tokenizador **de palabras** (whitespace).

### Conceptos clave
- **Vocabulario** $\mathcal{V}$: conjunto finito de tokens
- **IDs**: cada token tiene un entero
- **OOV**: palabras fuera del vocabulario → `<unk>` (si existe)

In [ ]:
from collections import Counter

@dataclass
class Vocab:
    stoi: Dict[str, int]
    itos: List[str]
    pad: str = '<pad>'
    unk: str = '<unk>'
    bos: str = '<bos>'
    eos: str = '<eos>'

    @property
    def pad_id(self): return self.stoi[self.pad]
    @property
    def unk_id(self): return self.stoi[self.unk]
    @property
    def bos_id(self): return self.stoi[self.bos]
    @property
    def eos_id(self): return self.stoi[self.eos]

def build_vocab(texts: List[str], min_freq: int = 1) -> Vocab:
    counter = Counter()
    for t in texts:
        counter.update(t.strip().split())
    specials = ['<pad>', '<unk>', '<bos>', '<eos>']
    itos = specials + [w for w,c in counter.items() if c >= min_freq and w not in specials]
    stoi = {w:i for i,w in enumerate(itos)}
    return Vocab(stoi=stoi, itos=itos)

def encode(vocab: Vocab, text: str, add_bos_eos: bool=False) -> List[int]:
    toks = text.strip().split()
    ids = [vocab.stoi.get(w, vocab.unk_id) for w in toks]
    if add_bos_eos:
        ids = [vocab.bos_id] + ids + [vocab.eos_id]
    return ids

def decode(vocab: Vocab, ids: List[int]) -> str:
    return ' '.join(vocab.itos[i] if 0 <= i < len(vocab.itos) else '<oov>' for i in ids)

def pad_batch(seqs: List[List[int]], pad_id: int) -> Tuple[torch.Tensor, torch.Tensor]:
    lengths = torch.tensor([len(s) for s in seqs], dtype=torch.long)
    T = int(lengths.max().item())
    out = torch.full((len(seqs), T), pad_id, dtype=torch.long)
    for i,s in enumerate(seqs):
        out[i, :len(s)] = torch.tensor(s, dtype=torch.long)
    return out, lengths

print('Tokenizer utils ready')

Tokenizer utils ready


### Dataset de juguete
Usaremos un mini-corpus para construir el vocabulario.

In [ ]:
corpus = [
    'Me gusta la pizza',
    'Me gusta aprender NLP',
    'La pizza gusta a mucha gente',
    'Aprender embeddings ayuda en NLP'
]

vocab = build_vocab(corpus, min_freq=1)
print('Vocab size:', len(vocab.itos))
print('Primeros tokens:', vocab.itos[:15])

Vocab size: 18
Primeros tokens: ['<pad>', '<unk>', '<bos>', '<eos>', 'Me', 'gusta', 'la', 'pizza', 'aprender', 'NLP', 'La', 'a', 'mucha', 'gente', 'Aprender']


## Ejercicio 1 (tokenización básica)
**Objetivo**: comprobar que entiendes vocab, `<unk>`, y `<bos>/<eos>`.

**TODO**:
1) Tokeniza la frase: `"Me gusta la electroencefalografista"`
2) Observa qué pasa con la palabra rara (OOV)
3) Repite con `add_bos_eos=True`


## 3) Por qué el padding es necesario
En un batch, las secuencias tienen longitudes distintas. Para formar un tensor rectangular `(B, T)` se usa `<pad>`.

**Error típico**:
```python
torch.tensor([[1,2,3],[1,2]])  # falla
```
porque las filas no tienen la misma longitud.

In [ ]:
seqs = [encode(vocab, s, add_bos_eos=True) for s in corpus]
print('Longitudes:', [len(s) for s in seqs])

x_pad, lengths = pad_batch(seqs, vocab.pad_id)
print('x_pad shape:', x_pad.shape)
print('x_pad:\n', x_pad)
print('lengths:', lengths.tolist())

Longitudes: [6, 6, 8, 7]
x_pad shape: torch.Size([4, 8])
x_pad:
 tensor([[ 2,  4,  5,  6,  7,  3,  0,  0],
        [ 2,  4,  5,  8,  9,  3,  0,  0],
        [ 2, 10,  7,  5, 11, 12, 13,  3],
        [ 2, 14, 15, 16, 17,  9,  3,  0]])
lengths: [6, 6, 8, 7]


## Ejercicio 2 (padding)
**TODO**:
1) Elige un ejemplo i
2) Recupera la secuencia original eliminando `<pad>` usando `lengths`
3) Decodifícala a texto

## 4) Embeddings: matriz entrenable + lookup
Una capa de embeddings es una matriz:
$$E \in \mathbb{R}^{|\mathcal{V}|\times d}$$
y el embedding de un token `i` es la fila `E[i]`.

En PyTorch: `nn.Embedding(V, d)`.

In [ ]:
d_model = 16
emb = nn.Embedding(num_embeddings=len(vocab.itos), embedding_dim=d_model, padding_idx=vocab.pad_id).to(device)

x = x_pad.to(device)  # (B,T)
y = emb(x)            # (B,T,d)
print('x:', x.shape, '-> y:', y.shape)

x: torch.Size([4, 8]) -> y: torch.Size([4, 8, 16])


### Lookup explícito (para entender qué hace Embedding)
Esto es equivalente conceptualmente a indexar una matriz.

In [ ]:
E = emb.weight  # (V,d)
token_id = x[0,0].item()
print('token:', vocab.itos[token_id], '| id:', token_id)
print('E[token_id] shape:', E[token_id].shape)
print('embedding igual a emb(x)[0,0]? ', torch.allclose(E[token_id], y[0,0]))

token: <bos> | id: 2
E[token_id] shape: torch.Size([16])
embedding igual a emb(x)[0,0]?  True


## 5) Interpretación geométrica: similitud coseno
En embeddings, tokens con contextos similares tienden a acabar cerca.
Aquí solo veremos la mecánica (estos embeddings aún son aleatorios).


In [ ]:
def cosine(u, v, eps=1e-8):
    return float((u @ v) / (u.norm()*v.norm() + eps))

w1, w2 = 'pizza', 'NLP'
id1 = vocab.stoi.get(w1, vocab.unk_id)
id2 = vocab.stoi.get(w2, vocab.unk_id)
print(w1, w2, 'cos:', cosine(emb.weight[id1].detach().cpu(), emb.weight[id2].detach().cpu()))

pizza NLP cos: -0.038911353796720505


## 6) Tokenización por subpalabras (BPE, WordPiece, Unigram, Byte-level BPE)
Hasta ahora usamos tokens = palabras (whitespace). Eso tiene problemas:
- vocab crece mucho
- OOV (palabras raras) aparecen

Los **subwords** resuelven esto segmentando palabras raras en piezas frecuentes.

En Colab instalamos librerías:
- `tokenizers` (HuggingFace): BPE, WordPiece, Byte-level BPE
- `sentencepiece`: Unigram LM

In [ ]:
!pip -q install tokenizers sentencepiece

In [ ]:
sub_corpus = [
    'Me gusta aprender NLP con PyTorch.',
    'Los tokenizadores BPE y WordPiece segmentan palabras.',
    'Unigram LM aprende probabilidades de subpalabras.',
    'Byte-level BPE trabaja a nivel de bytes (útil para cualquier texto).',
    'Transformers usan subword tokenization para manejar OOV.',
    'Hoy entrenamos BPE, WordPiece, Unigram y Byte-level BPE.'
]

with open('sub_corpus.txt','w',encoding='utf-8') as f:
    for line in sub_corpus:
        f.write(line+'\n')

print('Lines:', len(sub_corpus))
print(sub_corpus[0])

Lines: 6
Me gusta aprender NLP con PyTorch.


## 6.1) BPE (Byte Pair Encoding)
Idea: empezar con símbolos (caracteres) y **fusionar pares frecuentes** iterativamente.
Tokenización final: aplica las fusiones (greedy).

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tok_bpe = Tokenizer(BPE(unk_token='[UNK]'))
tok_bpe.pre_tokenizer = Whitespace()

trainer_bpe = BpeTrainer(vocab_size=250, min_frequency=1,
                         special_tokens=['[PAD]','[UNK]','[CLS]','[SEP]','[MASK]'])
tok_bpe.train(['sub_corpus.txt'], trainer_bpe)

text = 'electroencefalografista aprende NLP'
enc = tok_bpe.encode(text)
print('TEXT:', text)
print('TOKENS:', enc.tokens)
print('DECODE:', tok_bpe.decode(enc.ids))

TEXT: electroencefalografista aprende NLP
TOKENS: ['el', 'e', 'c', 't', 'r', 'o', 'en', 'ce', 'f', 'al', 'o', 'gra', 'f', 'i', 's', 'ta', 'aprende', 'NLP']
DECODE: el e c t r o en ce f al o gra f i s ta aprende NLP


## Ejercicio 4 (BPE)
**TODO**:
1) Tokeniza un texto con emojis y acentos
2) Observa si aparece `[UNK]`
3) Cambia `vocab_size` y mira cómo cambian los tokens


## 6.2) WordPiece
WordPiece se usa en BERT. En práctica, también aprende subwords con un criterio tipo verosimilitud.
Suele marcar subwords internos con un prefijo (p.ej. `##`).

In [ ]:
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer

tok_wp = Tokenizer(WordPiece(unk_token='[UNK]'))
tok_wp.pre_tokenizer = Whitespace()

trainer_wp = WordPieceTrainer(vocab_size=250, min_frequency=1,
                              special_tokens=['[PAD]','[UNK]','[CLS]','[SEP]','[MASK]'])
tok_wp.train(['sub_corpus.txt'], trainer_wp)

text = 'jugando aprendemos tokenizacion'
enc = tok_wp.encode(text)
print('TEXT:', text)
print('TOKENS:', enc.tokens)
print('DECODE:', tok_wp.decode(enc.ids))

TEXT: jugando aprendemos tokenizacion
TOKENS: ['j', '##u', '##g', '##an', '##d', '##o', 'aprende', '##mos', 'tokeniz', '##a', '##c', '##i', '##on']
DECODE: j ##u ##g ##an ##d ##o aprende ##mos tokeniz ##a ##c ##i ##on


## 6.3) Unigram LM (SentencePiece)
Unigram parte de un vocab grande y **prunea** tokens, manteniendo los que maximizan la probabilidad.
Puede haber varias segmentaciones; elige la más probable.

Usaremos `sentencepiece`.

In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(
    input='sub_corpus.txt',
    model_prefix='sp_unigram',
    vocab_size=83,
    model_type='unigram',
    character_coverage=1.0,
    bos_id=1, eos_id=2, pad_id=0, unk_id=3
)

sp = spm.SentencePieceProcessor()
sp.load('sp_unigram.model')

text = 'desconocido electroencefalografista'
pieces = sp.encode(text, out_type=str)
ids = sp.encode(text, out_type=int)
print('TEXT:', text)
print('PIECES:', pieces)
print('DECODE:', sp.decode(ids))

TEXT: desconocido electroencefalografista
PIECES: ['▁', 'de', 's', 'c', 'o', 'n', 'o', 'c', 'i', 'd', 'o', '▁', 'e', 'l', 'e', 'c', 't', 'r', 'o', 'e', 'n', 'c', 'e', 'f', 'al', 'o', 'g', 'ra', 'f', 'i', 's', 'ta']
DECODE: desconocido electroencefalografista


## 6.4) Byte-level BPE
Variante BPE donde los símbolos iniciales son **bytes (0..255)**.
Ventaja: cobertura total Unicode (casi nunca necesitas `<unk>`).

Lo entrenamos con `ByteLevelBPETokenizer` (HuggingFace).

In [ ]:
from tokenizers import ByteLevelBPETokenizer

tok_bbpe = ByteLevelBPETokenizer()
tok_bbpe.train(['sub_corpus.txt'], vocab_size=250, min_frequency=1,
               special_tokens=['<s>','<pad>','</s>','<unk>','<mask>'])

text = 'Byte-level BPE: 汉字, emojis 🙂, acentos áéíóú'
enc = tok_bbpe.encode(text)
print('TEXT:', text)
print('TOKENS (primeros 60):', enc.tokens[:60])
print('¿<unk> aparece?', '<unk>' in enc.tokens)
print('DECODE:', tok_bbpe.decode(enc.ids))

TEXT: Byte-level BPE: 汉字, emojis 🙂, acentos áéíóú
TOKENS (primeros 60): ['B', 'y', 't', 'e', '-', 'l', 'e', 'v', 'e', 'l', 'Ġ', 'B', 'P', 'E', ':', 'Ġ', 'æ', '±', 'ī', 'å', 'Ń', 'Ĺ', ',', 'Ġ', 'e', 'm', 'o', 'j', 'i', 's', 'Ġ', 'ð', 'Ł', 'Ļ', 'Ĥ', ',', 'Ġ', 'a', 'c', 'e', 'n', 't', 'o', 's', 'Ġ', 'Ã', '¡', 'Ã', '©', 'Ã', 'Ń', 'Ã', '³', 'Ã', 'º']
¿<unk> aparece? False
DECODE: Byte-level BPE: 汉字, emojis 🙂, acentos áéíóú


## Ejercicio 5 (comparación subword)
**Objetivo**: ver cómo cambia la longitud $T$ según el tokenizador.

**TODO**:
1) Define un texto con una palabra muy rara + emojis
2) Tokeniza con BPE / WordPiece / Unigram / Byte-level BPE
3) Compara el número de tokens (longitud)

> Pista: la longitud influye en coste de cómputo (Transformers ~ $O(T^2)$ en atención).

## 7) Embeddings posicionales (intuición)
En Transformers, la atención no sabe el orden por sí misma.
Se suma un vector posicional $p_t$ al embedding del token:
$$x_t = e_{i_t} + p_t.$$

Aquí implementamos **positional embeddings aprendibles** (más simple que sinusoidales).

In [ ]:
class TinyPositionalEmbedding(nn.Module):
    def __init__(self, max_len: int, d_model: int):
        super().__init__()
        self.pos = nn.Embedding(max_len, d_model)
    def forward(self, x):
        B,T = x.shape
        positions = torch.arange(T, device=x.device).unsqueeze(0).expand(B, T)
        return self.pos(positions)

max_len = 32
pos_emb = TinyPositionalEmbedding(max_len=max_len, d_model=d_model).to(device)

x = x_pad.to(device)
tok_vecs = emb(x)
pos_vecs = pos_emb(x)
x_in = tok_vecs + pos_vecs
print('tok_vecs:', tok_vecs.shape)
print('pos_vecs:', pos_vecs.shape)
print('input final:', x_in.shape)

tok_vecs: torch.Size([4, 8, 16])
pos_vecs: torch.Size([4, 8, 16])
input final: torch.Size([4, 8, 16])


## Ejercicio 6 (posiciones)
**TODO**:
1) Crea dos secuencias con los mismos tokens pero en orden distinto.
2) Sin posicional: embeddings token a token son los mismos.
3) Con posicional: la suma cambia (por la posición).

Esto ilustra por qué los Transformers necesitan información posicional (Que se explicará el día siguiente).

# Redes Recurrentes (RNN, LSTM, GRU) para PLN
Esta sección corresponde con la sección **“Las redes recurrentes para el procesamiento del lenguaje natural”**. de la teoría.

Donde vamos a explicar algunos detalles y teoria a nivel de código:
1) Repaso: tipos de datos y por qué MLP/CNN no son ideales para secuencias largas
2) Qué es una RNN (célula, estado, desenrollado)
3) **BPTT** en PyTorch (autograd)
4) **Vanishing/Exploding gradients** con un experimento controlado
5) Soluciones: **gradient clipping** y **truncated BPTT** (`detach()`)
6) LSTM y GRU: intuición, compuertas, y clasificación many-to-one

Hay **ejercicios intercalados (TODO)**.

In [ ]:
import math
import random
from dataclasses import dataclass
from typing import List, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
# Hacemos lo de antes, establecer semillas para reproducibilidad

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed = 42
set_seed(seed)
print('Seed set with value ', 42)

Seed set with value  42


## 1) Repaso: tipos de datos → sesgos inductivos
- **Tabular**: vector sin orden → MLP
- **Espacial (2D)**: localidad/traslación → CNN
- **Secuencial**: orden + dependencia temporal → RNN/Transformer

### Por qué MLP no es ideal en PLN
- Longitud fija
- No modela orden
- No comparte parámetros por posición

### Por qué CNN mejora pero sigue limitada
- Capta n-gramas (localidad)
- Paralelizable
- Pero el **contexto efectivo** crece lento (receptive field)
- Dependencias largas son difíciles de capturar de forma natural

## 2) ¿Qué es una RNN?
Una RNN es una **misma célula** aplicada repetidamente:

$$h_t = \tanh(W_x x_t + W_h h_{t-1} + b)$$

- $x_t$: entrada en el tiempo t
- $h_{t-1}$: memoria previa
- $h_t$: nueva memoria

La 'profundidad' de una RNN es **temporal**: el grafo se desenrolla a lo largo de $T$ pasos.

### Many-to-One (clasificación)
$$(x_1,\dots,x_T) \to y$$

### One-to-Many (generación)
$$x \to (y_1,\dots,y_T)$$

### Many-to-Many (etiquetado)
$$(x_1,\dots,x_T) \to (y_1,\dots,y_T)$$

## 3) Implementar una RNN desde cero (célula + desenrollado)
Construimos una célula RNN mínima y la aplicamos en un bucle.

Esto ayuda a ver:
- una sola célula reutilizada
- el estado conecta pasos
- autograd hará BPTT al final

In [ ]:
import torch
import torch.nn as nn

class SimpleRNNCell(nn.Module):
    """
    Una celda RNN "básica" (tipo Elman).
    Actualiza el estado oculto h_t usando:
        h_t = tanh(Wx * x_t + Wh * h_{t-1})
    """
    def __init__(self, input_dim: int, hidden_dim: int):
        super().__init__()

        # Proyección de la entrada x_t (D -> H) con bias
        self.Wx = nn.Linear(input_dim, hidden_dim, bias=True)

        # Proyección del estado anterior h_{t-1} (H -> H) sin bias
        # (muchas implementaciones ponen el bias solo en Wx)
        self.Wh = nn.Linear(hidden_dim, hidden_dim, bias=False)

    def forward(self, x_t, h_prev):
        """
        x_t:   (B, D)  entrada en el tiempo t
        h_prev:(B, H)  estado oculto anterior
        devuelve:
        h_t:   (B, H)  nuevo estado oculto
        """
        # Suma de contribución de la entrada y del estado anterior
        preact = self.Wx(x_t) + self.Wh(h_prev)

        # No linealidad típica de RNN simple
        h_t = torch.tanh(preact)
        return h_t


class SimpleRNN(nn.Module):
    """
    RNN que procesa una secuencia completa.
    Recorre T pasos, guarda todos los estados ocultos, y produce logits
    a partir del último estado (many-to-one).
    """
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int):
        super().__init__()

        # La celda recurrente (actualiza h en cada t)
        self.cell = SimpleRNNCell(input_dim, hidden_dim)

        # Capa final para convertir el último estado oculto en salida/clases
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None):
        """
        x:  (B, T, D) batch de secuencias
            B = batch size
            T = longitud temporal
            D = dimensión de entrada (input_dim)

        h0: (B, H) estado inicial opcional
        devuelve:
        logits: (B, output_dim) salida basada en el último estado
        hs:     (B, T, H) todos los estados ocultos a lo largo del tiempo
        """
        B, T, D = x.shape

        # Si no dan h0, inicializamos h a ceros (estado oculto inicial)
        if h0 is None:
            H = self.cell.Wh.in_features  # hidden_dim (entrada de Wh)
            h = torch.zeros(B, H, device=x.device)
        else:
            h = h0

        hs = []  # aquí guardaremos cada h_t

        # Recorremos la secuencia paso a paso
        for t in range(T):
            # Tomamos la entrada del tiempo t: (B, D)
            x_t = x[:, t, :]

            # Actualizamos el estado oculto: (B, H)
            h = self.cell(x_t, h)

            # Guardamos este estado
            hs.append(h)

        # Many-to-one: usamos SOLO el último estado oculto para predecir
        logits = self.fc(hs[-1])  # (B, output_dim)

        # Apilamos la lista hs -> tensor (B, T, H)
        hs = torch.stack(hs, dim=1)

        return logits, hs


print("SimpleRNN ready")


SimpleRNN ready


### Dataset de juguete: paridad (depende de todo el pasado)
Dada una secuencia de bits, predecir si la suma de 1s es **par** o **impar**.
Esto obliga a la red a mantener una memoria a lo largo de T pasos.

In [ ]:
class ParityDataset(Dataset):
    """
    Dataset para el problema de paridad.
    Cada muestra es una secuencia binaria y la etiqueta indica
    si el número total de unos es par (0) o impar (1).
    """
    def __init__(self, n_samples=2000, seq_len=20):
        self.n = n_samples   # número total de ejemplos
        self.T = seq_len    # longitud de cada secuencia

    def __len__(self):
        # Número de muestras del dataset
        return self.n

    def __getitem__(self, idx):
        # Genera una secuencia aleatoria de 0s y 1s
        # x: (T,)
        x = torch.randint(0, 2, (self.T,))

        # Etiqueta: paridad de la suma
        # sum % 2 = 0 → par, 1 → impar
        y = int(x.sum().item() % 2)

        # Convertimos a one-hot para usarlo como entrada a la red
        # (T,) → (T, 2)
        x_oh = F.one_hot(x, num_classes=2).float()

        return x_oh, y


def make_loader(seq_len=20, batch=64):
    """
    Crea un DataLoader listo para entrenar.
    """
    ds = ParityDataset(n_samples=2000, seq_len=seq_len)

    return DataLoader(
        ds,
        batch_size=batch,
        shuffle=True
    )


# Probamos el loader
dl = make_loader(seq_len=20, batch=64)

# Sacamos un batch
x, y = next(iter(dl))

print('x:', x.shape)        # (B, T, 2)
print('y:', y.shape)        # (B,)
print('x[0]', x[0].tolist())
print('y[:10]:', y[:10].tolist())

x: torch.Size([64, 20, 2])
y: torch.Size([64])
x[0] [[0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0]]
y[:10]: [0, 1, 0, 0, 1, 1, 0, 0, 1, 0]


## Ejercicio 1 (desenrollado)
**TODO**:
1) Ejecuta el modelo una vez
2) Imprime shapes de `logits` y `hs`
3) Explica qué representa `hs[:, t, :]`

## 4) Entrenamiento y BPTT (autograd)
En PyTorch, `loss.backward()` hace BPTT automáticamente.
Inspeccionamos loss y accuracy.

In [ ]:
import torch
import torch.nn as nn

def train_parity(model, dl, epochs=10, lr=1e-3, clip=None):
    # Ponemos el modelo en modo entrenamiento:
    # activa dropout/batchnorm (si existieran) y prepara el comportamiento correcto
    model.train()

    # Optimizador: Adam actualiza los parámetros del modelo usando los gradientes
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    # Función de pérdida para clasificación multiclase:
    # espera logits de shape (B, C) y etiquetas y de shape (B,) con valores 0..C-1
    crit = nn.CrossEntropyLoss()

    # Bucle de épocas (pasadas completas por el DataLoader)
    for ep in range(1, epochs + 1):
        total_loss, total_acc, n = 0.0, 0.0, 0  # acumuladores para métricas

        # Iteramos por mini-batches
        for x, y in dl:
            # x: (B, T, 2)  secuencia one-hot
            # y: (B,)       etiqueta (0=par, 1=impar)
            x, y = x.to(device), y.to(device)

            # 1) Resetear gradientes acumulados del paso anterior
            opt.zero_grad()

            # 2) Forward: obtenemos los logits (B, 2)
            #    El modelo devuelve (logits, hs). Aquí hs no lo usamos.
            logits, _ = model(x)

            # 3) Loss: CrossEntropy incluye softmax internamente
            loss = crit(logits, y)

            # 4) Backward: calcula gradientes (BPTT: backprop through time)
            loss.backward()

            # (Opcional) Clipping de gradientes para estabilizar entrenamiento en RNNs
            # Evita explosión de gradientes
            if clip is not None:
                nn.utils.clip_grad_norm_(model.parameters(), clip)

            # 5) Paso del optimizador: actualiza los parámetros con los gradientes
            opt.step()

            # --- Métricas ---
            # loss promedio ponderado por tamaño de batch
            total_loss += loss.item() * y.size(0)

            # accuracy: comparamos clase predicha (argmax) contra y
            preds = logits.argmax(dim=-1)  # (B,)
            total_acc += (preds == y).float().sum().item()

            # contador total de ejemplos vistos
            n += y.size(0)

        # Log cada 2 épocas (y también en la primera)
        if ep % 2 == 0 or ep == 1:
            print(f'ep {ep:02d} | loss {total_loss/n:.4f} | acc {total_acc/n:.3f}')


# Creamos y entrenamos
model = SimpleRNN(input_dim=2, hidden_dim=32, output_dim=2).to(device)
train_parity(model, dl, epochs=10, lr=1e-3, clip=None)


ep 01 | loss 0.6974 | acc 0.488
ep 02 | loss 0.6961 | acc 0.495
ep 04 | loss 0.6954 | acc 0.476
ep 06 | loss 0.6942 | acc 0.521
ep 08 | loss 0.6944 | acc 0.500
ep 10 | loss 0.6939 | acc 0.491


Qué significa “BPTT” aquí

Tu RNN guarda la cadena de operaciones para todos los timesteps t=0..T-1.
Cuando haces loss.backward(), PyTorch propaga gradientes desde el último estado hacia atrás por toda la secuencia.

## Ejercicio 2 (gradientes)
**TODO**:
1) Haz forward+backward con un batch
2) Imprime norma de gradientes de `Wx` y `Wh`
3) Explica por qué `Wh` es crítico en dependencias largas

## 5) Vanishing/Exploding gradient: experimento controlado
Simplificación:
$$\frac{\partial h_t}{\partial h_{t-k}} \approx (W_h)^k$$
Si la norma efectiva es <1 → desaparece; >1 → explota.

Aquí definimos: $h_t = \tanh(\alpha h_{t-1})$ y medimos $\|\partial L/\partial h_0\|$.

In [ ]:
class ControlledRNNCell(nn.Module):
    """
    Celda RNN artificial MUY simple.
    No tiene pesos aprendibles: solo controla cuánto del estado anterior
    se propaga al siguiente paso mediante el escalar alpha.
    """
    def __init__(self, hidden_dim: int, alpha: float):
        super().__init__()
        self.alpha = alpha          # factor de escala (controla estabilidad)
        self.hidden_dim = hidden_dim

    def forward(self, h_prev):
        # Regla de transición:
        # h_t = tanh(alpha * h_{t-1})
        return torch.tanh(self.alpha * h_prev)

def grad_through_time(alpha: float, T: int = 50, hidden_dim: int = 64):
    # Creamos la celda con un alpha fijo
    cell = ControlledRNNCell(hidden_dim, alpha).to(device)

    # Estado inicial h0, con gradiente activado
    # shape: (1, hidden_dim)
    h0 = torch.randn(1, hidden_dim, device=device, requires_grad=True)

    # Inicializamos el estado
    h = h0

    # Avanzamos T pasos en el tiempo (sin entrada externa)
    for _ in range(T):
        h = cell(h)

    # Definimos una pérdida artificial: suma de todas las componentes finales
    loss = h.sum()

    # Backpropagation Through Time
    loss.backward()

    # Devolvemos la norma del gradiente respecto al estado inicial
    return float(h0.grad.norm().item())

for a in [0.5, 0.9, 1.0, 1.1, 1.5]:
    g = grad_through_time(alpha=a, T=60)
    print(f'alpha={a:3.1f} -> ||dL/dh0|| = {g:.6e}')

alpha=0.5 -> ||dL/dh0|| = 5.507782e-18
alpha=0.9 -> ||dL/dh0|| = 8.623534e-03
alpha=1.0 -> ||dL/dh0|| = 3.017680e+00
alpha=1.1 -> ||dL/dh0|| = 6.628270e+00
alpha=1.5 -> ||dL/dh0|| = 1.254204e-20


## Ejercicio 3 (efecto de T)
**TODO**: prueba T=10,30,100 en alpha=0.9 y alpha=1.1 y explica el patrón.

## 6) Solución para exploding: Gradient clipping
Mostramos entrenamiento con y sin clipping en secuencias más largas.

In [ ]:
# DataLoader con secuencias más largas (T=60).
# Esto hace el problema más difícil para una RNN simple y aumenta el riesgo
# de vanishing/exploding gradients.
dl_long = make_loader(seq_len=60, batch=64)

# Dos modelos idénticos para comparar de forma justa:
# - uno sin clipping
# - otro con clipping
model_no_clip = SimpleRNN(input_dim=2, hidden_dim=64, output_dim=2).to(device)
model_clip    = SimpleRNN(input_dim=2, hidden_dim=64, output_dim=2).to(device)

print('--- Sin clipping ---')
# Entrenamos 6 épocas con lr=3e-3 (algo más alto que 1e-3),
# y SIN recortar gradientes.
train_parity(model_no_clip, dl_long, epochs=6, lr=3e-3, clip=None)

print('\n--- Con clipping (1.0) ---')
# Entrenamos 6 épocas con el mismo lr,
# pero ahora recortando la norma global de gradientes a 1.0.
# Esto ayuda a evitar pasos gigantes cuando los gradientes explotan.
train_parity(model_clip, dl_long, epochs=6, lr=3e-3, clip=1.0)


--- Sin clipping ---
ep 01 | loss 0.6984 | acc 0.510
ep 02 | loss 0.7032 | acc 0.490
ep 04 | loss 0.6979 | acc 0.494
ep 06 | loss 0.6960 | acc 0.492

--- Con clipping (1.0) ---
ep 01 | loss 0.6985 | acc 0.520
ep 02 | loss 0.7064 | acc 0.504
ep 04 | loss 0.6971 | acc 0.512
ep 06 | loss 0.6948 | acc 0.523


## 7) Truncated BPTT + detach()
Procesamos la secuencia por chunks y cortamos el grafo para limitar cuánto atrás viaja el gradiente.

In [ ]:
def truncated_bptt_last_chunk_grad(T=80, chunk=20, hidden_dim=64, alpha=0.9):
    cell = ControlledRNNCell(hidden_dim, alpha).to(device)

    # Estado inicial (leaf)
    h = torch.randn(1, hidden_dim, device=device, requires_grad=True)

    # Iremos guardando aquí el "inicio del último chunk" como leaf
    h_start_last_leaf = None

    # El último chunk empieza en el paso: T - chunk (0-indexed)
    last_start_step = T - chunk  # ej: T=80, chunk=20 -> empieza en t=60

    for t in range(T):
        # Si estamos justo al inicio del último chunk,
        # hacemos detach para cortar el grafo ANTES de entrar al último chunk
        # y convertimos h en un leaf con requires_grad=True
        if t == last_start_step:
            h = h.detach().requires_grad_(True)
            h_start_last_leaf = h  # este SÍ es leaf

        # Avanzamos un paso
        h = cell(h)

        # Truncamiento normal cada 'chunk' pasos (excepto al final)
        if (t + 1) % chunk == 0 and (t + 1) != T:
            h = h.detach().requires_grad_(True)

    loss = h.sum()
    loss.backward()

    # Si chunk == T, last_start_step = 0, así que h_start_last_leaf se define bien
    return float(h_start_last_leaf.grad.norm().item())


for chunk in [10, 20, 40, 80]:
    g = truncated_bptt_last_chunk_grad(T=80, chunk=chunk, alpha=0.9)
    print(f'chunk={chunk:3d} -> ||dL/dh_start_last|| = {g:.6e}')


chunk= 10 -> ||dL/dh_start_last|| = 2.789426e+00
chunk= 20 -> ||dL/dh_start_last|| = 9.726100e-01
chunk= 40 -> ||dL/dh_start_last|| = 1.182283e-01
chunk= 80 -> ||dL/dh_start_last|| = 8.430752e-04


## 8) LSTM y GRU: clasificación many-to-one
Ahora pasamos al caso PLN típico:
**Embedding → (LSTM/GRU) → último estado → Linear → clase**

Usamos tokenización whitespace para no mezclar temas.

In [ ]:
from collections import Counter
from dataclasses import dataclass
import torch
from torch.utils.data import Dataset, DataLoader

@dataclass
class Vocab:
    stoi: dict   # string -> id
    itos: list   # id -> string
    pad: str = '<pad>'
    unk: str = '<unk>'
    bos: str = '<bos>'
    eos: str = '<eos>'

    # IDs de tokens especiales
    @property
    def pad_id(self): return self.stoi[self.pad]
    @property
    def unk_id(self): return self.stoi[self.unk]
    @property
    def bos_id(self): return self.stoi[self.bos]
    @property
    def eos_id(self): return self.stoi[self.eos]

def build_vocab(texts, min_freq=1):
    # Cuenta palabras en todos los textos
    c = Counter()
    for t in texts:
        c.update(t.lower().split())

    # Tokens especiales primero (quedan con ids 0..3)
    specials = ['<pad>', '<unk>', '<bos>', '<eos>']

    # Añadimos el resto de palabras que aparecen al menos min_freq veces
    itos = specials + [w for w, f in c.items() if f >= min_freq and w not in specials]
    stoi = {w: i for i, w in enumerate(itos)}

    return Vocab(stoi, itos)

def encode(v: Vocab, text: str, add_bos_eos=True):
    # Tokenización simple por espacios (para demo)
    toks = text.lower().split()

    # Convertimos tokens a ids (si no está, usamos unk_id)
    ids = [v.stoi.get(w, v.unk_id) for w in toks]

    # Añadimos tokens de inicio y fin si se pide
    if add_bos_eos:
        ids = [v.bos_id] + ids + [v.eos_id]
    return ids

def pad_batch(seqs, pad_id):
    # Longitudes reales de cada secuencia
    lens = torch.tensor([len(s) for s in seqs], dtype=torch.long)

    # Longitud máxima en este batch
    T = int(lens.max().item())

    # Matriz (B, T) rellena con pad_id
    out = torch.full((len(seqs), T), pad_id, dtype=torch.long)

    # Copiamos cada secuencia al principio (padding a la derecha)
    for i, s in enumerate(seqs):
        out[i, :len(s)] = torch.tensor(s, dtype=torch.long)

    return out, lens

# Datos de ejemplo
texts = [
    'el perro que persiguio al gato estaba cansado',
    'me gusta la pizza',
    'pytorch entrena redes recurrentes',
    'hoy hace sol en la playa',
    'la rnn mantiene memoria del pasado',
    'cnn capta patrones locales',
    'me encanta aprender nlp',
    'el banco del parque es verde',
    'el banco central sube tipos',
    'las lstm usan compuertas'
]
labels = [1,1,0,1,0,0,0,1,0,0]

# Construimos vocabulario con estos textos
v = build_vocab(texts)

class TextCls(Dataset):
    # Dataset que devuelve (secuencia_ids, label)
    def __init__(self, texts, labels, v):
        self.texts = texts
        self.labels = labels
        self.v = v

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, i):
        # ids = lista de ints (longitud variable)
        ids = encode(self.v, self.texts[i], add_bos_eos=True)
        y = int(self.labels[i])
        return ids, y

def collate(batch):
    # batch es una lista de elementos devueltos por el Dataset
    # cada uno: (lista_ids, y)
    seqs, ys = zip(*batch)

    # Pad a la longitud máxima del batch
    x, l = pad_batch(list(seqs), v.pad_id)

    # Convertimos labels a tensor
    y = torch.tensor(ys, dtype=torch.long)
    return x, l, y

# DataLoader con collate_fn para padding dinámico por batch
dl_text = DataLoader(
    TextCls(texts, labels, v),
    batch_size=4,
    shuffle=True,
    collate_fn=collate
)

# Probamos un batch
xb, lb, yb = next(iter(dl_text))
print('xb:', xb.shape, 'lengths:', lb.tolist(), 'y:', yb.tolist())


xb: torch.Size([4, 10]) lengths: [6, 10, 7, 6] y: [0, 1, 0, 0]


In [ ]:
import torch
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, d_model=64, hidden=128, num_classes=2, pad_id=0):
        super().__init__()

        # Embedding: convierte ids (0..vocab_size-1) en vectores de tamaño d_model.
        # padding_idx=pad_id hace que:
        # - la fila del PAD se mantenga fija (no se actualiza)
        # - su gradiente sea 0
        self.emb = nn.Embedding(vocab_size, d_model, padding_idx=pad_id)

        # LSTM (batch_first=True => entrada (B, T, d_model))
        self.lstm = nn.LSTM(d_model, hidden, batch_first=True)

        # Capa final de clasificación
        self.fc = nn.Linear(hidden, num_classes)

    def forward(self, x, lengths):
        """
        x:       (B, T) ids con padding
        lengths: (B,) longitudes reales (sin contar el padding)
        devuelve:
        logits:  (B, num_classes)
        """
        # (B, T) -> (B, T, d_model)
        e = self.emb(x)

        # Empaquetado para ignorar pasos PAD.
        # enforce_sorted=False permite que el batch no esté ordenado por longitud.
        packed = nn.utils.rnn.pack_padded_sequence(
            e, lengths.cpu(), batch_first=True, enforce_sorted=False
        )

        # Pasamos el packed por la LSTM.
        # Para LSTM, la salida final es (hT, cT):
        # hT: (num_layers * num_directions, B, hidden)
        # cT: (num_layers * num_directions, B, hidden)
        _, (hT, cT) = self.lstm(packed)

        # Usamos el hidden final de la última capa: hT[-1] -> (B, hidden)
        # y lo proyectamos a clases: (B, 2)
        logits = self.fc(hT[-1])
        return logits


class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, d_model=64, hidden=128, num_classes=2, pad_id=0):
        super().__init__()

        # Igual: embedding con padding_idx
        self.emb = nn.Embedding(vocab_size, d_model, padding_idx=pad_id)

        # GRU (similar a LSTM pero sin cT)
        self.gru = nn.GRU(d_model, hidden, batch_first=True)

        # Clasificador final
        self.fc = nn.Linear(hidden, num_classes)

    def forward(self, x, lengths):
        # (B, T) -> (B, T, d_model)
        e = self.emb(x)

        # Empaquetado para ignorar padding
        packed = nn.utils.rnn.pack_padded_sequence(
            e, lengths.cpu(), batch_first=True, enforce_sorted=False
        )

        # Para GRU, devuelve:
        # hT: (num_layers * num_directions, B, hidden)
        _, hT = self.gru(packed)

        # Última capa: (B, hidden) -> logits (B, 2)
        logits = self.fc(hT[-1])
        return logits


def train_cls(model, dl, epochs=25, lr=2e-3, clip=1.0):
    # Mandamos el modelo al device (CPU/GPU)
    model.to(device)

    # Optimizador
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    # Loss para clasificación
    crit = nn.CrossEntropyLoss()

    for ep in range(1, epochs + 1):
        model.train()

        tot_loss, tot_acc, n = 0.0, 0.0, 0

        for x, l, y in dl:
            # x: (B, T)  ids padded
            # l: (B,)    longitudes reales
            # y: (B,)    etiquetas
            x, l, y = x.to(device), l.to(device), y.to(device)

            # Reset gradientes
            opt.zero_grad()

            # Forward: logits (B, 2)
            logits = model(x, l)

            # Loss
            loss = crit(logits, y)

            # Backward
            loss.backward()

            # Clipping: evita exploding gradients
            if clip is not None:
                nn.utils.clip_grad_norm_(model.parameters(), clip)

            # Paso del optimizador
            opt.step()

            # Métricas
            tot_loss += loss.item() * y.size(0)
            tot_acc  += (logits.argmax(-1) == y).float().sum().item()
            n += y.size(0)

        # Print cada 5 epochs (y la primera)
        if ep % 5 == 0 or ep == 1:
            print(f'ep {ep:02d} | loss {tot_loss/n:.4f} | acc {tot_acc/n:.3f}')


print('--- LSTM ---')
train_cls(LSTMClassifier(len(v.itos), pad_id=v.pad_id), dl_text)

print('\n--- GRU ---')
train_cls(GRUClassifier(len(v.itos), pad_id=v.pad_id), dl_text)


--- LSTM ---
ep 01 | loss 0.7291 | acc 0.400
ep 05 | loss 0.2839 | acc 1.000
ep 10 | loss 0.0014 | acc 1.000
ep 15 | loss 0.0002 | acc 1.000
ep 20 | loss 0.0001 | acc 1.000
ep 25 | loss 0.0001 | acc 1.000

--- GRU ---
ep 01 | loss 0.7211 | acc 0.600
ep 05 | loss 0.1435 | acc 1.000
ep 10 | loss 0.0007 | acc 1.000
ep 15 | loss 0.0001 | acc 1.000
ep 20 | loss 0.0001 | acc 1.000
ep 25 | loss 0.0001 | acc 1.000


## Ejercicio 4 (many-to-one en texto)
**TODO**:
1) Añade 5 frases nuevas por clase
2) Re-entrena y compara LSTM vs GRU
3) Quita `pack_padded_sequence` y observa cambios
4) Explica por qué el padding puede sesgar el resumen si no se maneja


El padding puede sesgar el resumen porque, si no se maneja explícitamente, la RNN sigue actualizando su estado oculto durante los timesteps <pad>.
En un esquema many-to-one que toma el último estado del batch, ese estado puede corresponder a padding y no al final real de la secuencia.
Aunque el embedding del PAD esté congelado, la dinámica recurrente sigue modificando el estado y el modelo puede aprender a usar la cantidad de padding (longitud) como pista espuria.
En datasets pequeños esto puede no afectar la accuracy, pero en problemas reales perjudica la generalización.

# Modelos de lenguaje autorregresivos (RNN/LSTM)
Esta sección implementa y explica, paso a paso:
- Qué es un **modelo de lenguaje** y por qué asigna probabilidades a secuencias
- Factorización **autorregresiva**: $P(w_{1:T}) = \prod_t P(w_t\mid w_{<t})$
- Entrenamiento con **teacher forcing** (shift input/target)
- Implementación de un **LSTM Language Model** en PyTorch
- **Generación** token a token (argmax / sampling / temperature)
- Evaluación con **perplejidad (PPL)**

¡Incluye ejercicios intercalados (TODO)!

In [ ]:
import math
import random
from dataclasses import dataclass
from typing import List, Dict, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)
print('Seed set')

Seed set


## 1) ¿Qué es un modelo de lenguaje?
Un modelo de lenguaje asigna probabilidades a secuencias:
$$P(w_1,\dots,w_T).$$

Intuición: completa huecos *"Hoy hace mucho ___"*.

## 2) Autorregresivo
Un modelo autorregresivo predice la siguiente palabra usando el pasado:
$$P(w_1,\dots,w_T)=\prod_{t=1}^{T}P(w_t\mid w_{<t}).$$

Esto permite **generación**: el modelo se alimenta de sus predicciones.

## 3) Preparar datos: tokenización + IDs
Para centrarnos en el modelo, usaremos tokenización whitespace y un vocabulario pequeño.

Incluimos tokens especiales:
- `<bos>` inicio
- `<eos>` fin
- `<pad>` padding (si hacemos batches de secuencias)
- `<unk>` OOV

En un LM autorregresivo entrenamos con:
- input: `[w1,...,w(T-1)]`
- target: `[w2,...,wT]` (shift)

In [ ]:
from collections import Counter
from dataclasses import dataclass
from typing import Dict, List

@dataclass
class Vocab:
    # Mapa palabra -> id
    stoi: Dict[str, int]

    # Mapa id -> palabra
    itos: List[str]

    # Tokens especiales
    pad: str = '<pad>'
    unk: str = '<unk>'
    bos: str = '<bos>'
    eos: str = '<eos>'

    # Accesos rápidos a los ids de los tokens especiales
    @property
    def pad_id(self): return self.stoi[self.pad]

    @property
    def unk_id(self): return self.stoi[self.unk]

    @property
    def bos_id(self): return self.stoi[self.bos]

    @property
    def eos_id(self): return self.stoi[self.eos]


def build_vocab(texts: List[str], min_freq: int = 1) -> Vocab:
    """
    Construye un vocabulario a partir de una lista de textos.
    """
    # Contador de frecuencias de palabras
    c = Counter()
    for t in texts:
        c.update(t.lower().split())

    # Tokens especiales (siempre primero)
    specials = ['<pad>', '<unk>', '<bos>', '<eos>']

    # Lista id -> token
    # Incluimos palabras que aparecen al menos min_freq veces
    itos = specials + [
        w for w, f in c.items()
        if f >= min_freq and w not in specials
    ]

    # Diccionario token -> id
    stoi = {w: i for i, w in enumerate(itos)}

    return Vocab(stoi=stoi, itos=itos)


def encode(v: Vocab, text: str, add_bos_eos: bool = True) -> List[int]:
    """
    Convierte un texto en una lista de ids.
    """
    # Tokenización simple
    toks = text.lower().split()

    # Mapeo palabra -> id (unk si no existe)
    ids = [v.stoi.get(w, v.unk_id) for w in toks]

    # Añadimos BOS y EOS si se pide
    if add_bos_eos:
        ids = [v.bos_id] + ids + [v.eos_id]

    return ids


def decode(v: Vocab, ids: List[int]) -> str:
    """
    Convierte una lista de ids de vuelta a texto (para debug).
    """
    return ' '.join(
        v.itos[i] if 0 <= i < len(v.itos) else '<oov>'
        for i in ids
    )


print('Vocab utils ready')


Vocab utils ready


In [ ]:
corpus = [
    'hoy hace mucho frio en madrid',
    'hoy hace mucho calor en sevilla',
    'el gato duerme en el sofa',
    'el perro duerme en el suelo',
    'me gusta aprender nlp con pytorch',
    'las rnn predicen la siguiente palabra',
    'las lstm usan compuertas para recordar',
    'los modelos autorregresivos generan texto',
    'el gradiente puede desaparecer o explotar',
    'hacemos teacher forcing durante el entrenamiento',
]

vocab = build_vocab(corpus, min_freq=1)
print('Vocab size:', len(vocab.itos))
print('Ejemplo vocab:', vocab.itos[:20])

Vocab size: 50
Ejemplo vocab: ['<pad>', '<unk>', '<bos>', '<eos>', 'hoy', 'hace', 'mucho', 'frio', 'en', 'madrid', 'calor', 'sevilla', 'el', 'gato', 'duerme', 'sofa', 'perro', 'suelo', 'me', 'gusta']


## Ejercicio 1 (factorización)
**TODO**: en tus palabras, explica qué significa:
$$P(w_1,\dots,w_T)=\prod_t P(w_t\mid w_{<t}).$$

Luego, responde:
1) ¿Por qué esto permite generación?
2) ¿Qué información usa el modelo para predecir $w_t$?

> Solución esperada: usa contexto previo, palabra a palabra, y al generar reutiliza predicciones como entrada.

## 4) Dataset autorregresivo (shift)
Vamos a convertir el corpus en una secuencia larga de IDs y crear pares (x,y) por ventanas.

Esto es típico para LM con RNN: **truncated BPTT** implícito por ventanas.

- `block_size = T` define cuántos tokens de contexto usamos.
- `x = ids[i:i+T]`
- `y = ids[i+1:i+T+1]`

In [ ]:
# Lista donde acumularemos TODOS los tokens del corpus
all_ids = []

# Recorremos cada frase del corpus
for s in corpus:
    # Codificamos la frase:
    # texto -> ids, añadiendo <bos> y <eos>
    ids = encode(vocab, s, add_bos_eos=True)

    # Añadimos esos ids a la lista global (aplanado)
    all_ids.extend(ids)

# Convertimos la lista a tensor (necesario para PyTorch)
all_ids = torch.tensor(all_ids, dtype=torch.long)

# Número total de tokens (incluye <bos> y <eos>)
print('Total tokens:', len(all_ids))

# Decodificamos algunos tokens para comprobar que todo está correcto
print('Decode sample:', decode(vocab, all_ids[:12].tolist()))


Total tokens: 79
Decode sample: <bos> hoy hace mucho frio en madrid <eos> <bos> hoy hace mucho


## Ejercicios y recurso del uso de RNN como LSTM/GRU en PLN: Modelo de lenguaje (siguiente palabra) con GRU → cámbialo a LSTM y compara

### Objetivo
Ahora practicamos un modelo de **lenguaje autorregresivo** (next-token prediction) usando un dataset público real:** WikiText-2.**

- Entrada: `x = (w1, w2, ..., w_{T-1})`
- Target: `y = (w2, w3, ..., w_T)`
- La red produce logits **en cada paso temporal**, y entrenamos con `CrossEntropyLoss` token a token.
- Se evalúa con: **Negative Log-Likelihood (NLL)** y **Perplexity (PPL)**

### Dataset:
Utilizaremos
**WikiText-2 (raw)**: https://huggingface.co/datasets/Salesforce/wikitext (usamores los 2500 primeros textos de train para el entrenamiento y 500 de validación por la limitación de GPU)

### Arquitectura del modelo
Embedding → RNN (GRU o LSTM) → Linear → Softmax

### Qué tienes que hacer (TODO)
1. Entrena el modelo base **GRU** (ya dado) y anota `loss` y `perplexity`.
2. Duplica el modelo y sustitúyelo por **LSTM**.
3. Entrena con los **mismos hiperparámetros** (epochs, lr, hidden, etc.).
4. Compara: ¿mejora la `perplexity`? ¿se entrena más estable?



In [ ]:
import math
import random
from collections import Counter
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset

# -------------------------
# 1) Vocab + encode/decode
# -------------------------
@dataclass
class VocabLM:
    stoi: Dict[str, int]
    itos: List[str]
    pad: str = "<pad>"
    unk: str = "<unk>"
    bos: str = "<bos>"
    eos: str = "<eos>"

    @property
    def pad_id(self): return self.stoi[self.pad]
    @property
    def unk_id(self): return self.stoi[self.unk]
    @property
    def bos_id(self): return self.stoi[self.bos]
    @property
    def eos_id(self): return self.stoi[self.eos]


def build_vocab_lm(texts: List[str], min_freq: int = 2, max_vocab: int = 50000) -> VocabLM:
    c = Counter()
    for t in texts:
        c.update(t.lower().split())

    specials = ["<pad>", "<unk>", "<bos>", "<eos>"]
    # ordenamos por frecuencia (desc) y luego lexicográfico para estabilidad
    words = sorted([(w, f) for w, f in c.items() if w not in specials and f >= min_freq],
                   key=lambda x: (-x[1], x[0]))
    words = [w for w, _ in words][: max(0, max_vocab - len(specials))]

    itos = specials + words
    stoi = {w: i for i, w in enumerate(itos)}
    return VocabLM(stoi=stoi, itos=itos)


def encode_lm(v: VocabLM, text: str, add_bos_eos: bool = True) -> List[int]:
    toks = text.lower().split()
    ids = [v.stoi.get(w, v.unk_id) for w in toks]
    if add_bos_eos:
        ids = [v.bos_id] + ids + [v.eos_id]
    return ids


def decode_lm(v: VocabLM, ids: List[int], skip_specials: bool = True) -> str:
    specials = {v.pad, v.unk, v.bos, v.eos}
    toks = []
    for i in ids:
        w = v.itos[i] if 0 <= i < len(v.itos) else v.unk
        if skip_specials and w in specials:
            continue
        toks.append(w)
    return " ".join(toks)

In [ ]:
# -------------------------
# 2) Dataset LM por ventanas
# -------------------------
class LMDataset(Dataset):
    def __init__(self, ids: torch.Tensor, block_size: int):
        self.ids = ids
        self.block = block_size

    def __len__(self):
        return max(0, len(self.ids) - self.block - 1)

    def __getitem__(self, idx):
        x = self.ids[idx: idx + self.block]           # (T,)
        y = self.ids[idx + 1: idx + self.block + 1]   # (T,)
        return x, y

In [ ]:

# -------------------------
# 3) Modelo: Emb + (GRU|LSTM) + Linear
# -------------------------
class RNNLM(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        emb_dim: int = 128,
        hidden_dim: int = 256,
        num_layers: int = 1,
        rnn_type: str = "gru",      # "gru" o "lstm"
        dropout: float = 0.0,
        pad_id: int = 0
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_id)

        rnn_type = rnn_type.lower()
        self.rnn_type = rnn_type
        if rnn_type == "gru":
            self.rnn = nn.GRU(
                input_size=emb_dim,
                hidden_size=hidden_dim,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout if num_layers > 1 else 0.0
            )
        else:
            raise ValueError("rnn_type debe ser 'gru' o 'lstm'")

        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x: torch.Tensor, h=None):
        # x: (B, T)
        e = self.emb(x)             # (B, T, E)
        out, h = self.rnn(e, h)     # out: (B, T, H)
        logits = self.fc(out)       # (B, T, V)
        return logits, h


## Entrenamiento autorregresivo (teacher forcing)
Entrenamiento típico:
- `x` ya es contexto
- `y` es el target
- Loss: CrossEntropy sobre todos los pasos

Medimos también **perplejidad**:
$$\mathrm{PPL} = \exp(\text{loss promedio})$$
cuando loss es NLL por token.

### Modelo base (GRU) - EJEMPLO


In [ ]:
# -------------------------
# 4) Train/Eval (loss + ppl)
# -------------------------
@torch.no_grad()
def evaluate(model: nn.Module, dl: DataLoader, device: str) -> Tuple[float, float]:
    model.eval()
    total_loss = 0.0
    total_tokens = 0

    for x, y in dl:
        x, y = x.to(device), y.to(device)              # (B,T)
        logits, _ = model(x)                           # (B,T,V)
        loss = F.cross_entropy(
            logits.reshape(-1, logits.size(-1)),
            y.reshape(-1),
            reduction="sum"
        )
        total_loss += loss.item()
        total_tokens += y.numel()

    avg_nll = total_loss / max(1, total_tokens)       # NLL por token
    ppl = math.exp(min(20, avg_nll))                   # cap para evitar inf
    return avg_nll, ppl


def train_one_model(
    rnn_type: str,
    train_dl: DataLoader,
    valid_dl: DataLoader,
    vocab_size: int,
    pad_id: int,
    device: str,
    emb_dim: int = 128,
    hidden_dim: int = 256,
    num_layers: int = 1,
    dropout: float = 0.0,
    lr: float = 2e-3,
    epochs: int = 5,
    grad_clip: float = 1.0,
    seed: int = 123
):
    torch.manual_seed(seed)
    random.seed(seed)

    model = RNNLM(
        vocab_size=vocab_size,
        emb_dim=emb_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        rnn_type=rnn_type,
        dropout=dropout,
        pad_id=pad_id
    ).to(device)

    opt = torch.optim.AdamW(model.parameters(), lr=lr)

    for ep in range(1, epochs + 1):
        model.train()
        running = 0.0
        tokens = 0

        for x, y in train_dl:
            x, y = x.to(device), y.to(device)

            logits, _ = model(x)
            loss = F.cross_entropy(
                logits.reshape(-1, logits.size(-1)),
                y.reshape(-1),
                reduction="mean"
            )

            opt.zero_grad(set_to_none=True)
            loss.backward()
            if grad_clip is not None:
                nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            opt.step()

            running += loss.item() * y.numel()
            tokens += y.numel()

        train_nll = running / max(1, tokens)
        train_ppl = math.exp(min(20, train_nll))

        val_nll, val_ppl = evaluate(model, valid_dl, device)

        print(f"[{rnn_type.upper()}] epoch {ep:02d} | "
              f"train_nll {train_nll:.4f} ppl {train_ppl:.2f} | "
              f"valid_nll {val_nll:.4f} ppl {val_ppl:.2f}")

    return model

In [ ]:
# -------------------------
# 5) Inferencia: generar texto
# -------------------------
@torch.no_grad()
def generate_next_words(
    model: nn.Module,
    vocab: VocabLM,
    prompt: str,
    max_new_tokens: int = 20,
    temperature: float = 1.0,
    top_k: Optional[int] = 50,
    device: str = "cpu"
) -> str:
    """
    Generación autoregresiva:
    - Codifica el prompt (con <bos> ... sin <eos> al final)
    - Va prediciendo token a token
    """
    model.eval()

    # ids iniciales: <bos> + tokens prompt (sin eos)
    prompt_ids = [vocab.bos_id] + [vocab.stoi.get(w, vocab.unk_id) for w in prompt.lower().split()]
    ids = torch.tensor(prompt_ids, dtype=torch.long, device=device).unsqueeze(0)  # (1, T)

    h = None
    # "Warm-up" pasando todo el prompt para obtener estado oculto
    logits, h = model(ids, h)

    last_id = ids[:, -1:]  # (1,1)

    generated = prompt_ids[:]  # copia

    for _ in range(max_new_tokens):
        logits, h = model(last_id, h)             # (1,1,V)
        next_logits = logits[:, -1, :]            # (1,V)
        next_logits = next_logits / max(1e-6, temperature)

        if top_k is not None and top_k > 0:
            v, ix = torch.topk(next_logits, k=min(top_k, next_logits.size(-1)))
            mask = torch.full_like(next_logits, float("-inf"))
            mask.scatter_(1, ix, v)
            next_logits = mask

        probs = F.softmax(next_logits, dim=-1)    # (1,V)
        next_id = torch.multinomial(probs, num_samples=1)  # (1,1)

        tid = int(next_id.item())
        generated.append(tid)

        last_id = next_id
        if tid == vocab.eos_id:
            break

    return decode_lm(vocab, generated, skip_specials=True)


In [ ]:
def train_and_inference():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Device:", device)

    # Dataset público más grande
    ds = load_dataset("wikitext", "wikitext-2-raw-v1")

    # Usamos train para vocab; valid para evaluación
    train_texts_all = [t for t in ds["train"]["text"] if t.strip()]
    train_texts = train_texts_all[:2500]
    valid_texts_all = [t for t in ds["validation"]["text"] if t.strip()]
    valid_texts = valid_texts_all[:500]

    print("Train size:", len(train_texts))
    print("Valid size:", len(valid_texts))

    # Vocab (whitespace). Puedes subir min_freq o max_vocab
    vocab = build_vocab_lm(train_texts, min_freq=2, max_vocab=30000)
    print("Vocab size:", len(vocab.itos))

    # Convertimos a stream continuo de ids (autorregresivo)
    def texts_to_stream(texts: List[str]) -> torch.Tensor:
        all_ids = []
        for t in texts:
            all_ids.extend(encode_lm(vocab, t, add_bos_eos=True))
        return torch.tensor(all_ids, dtype=torch.long)

    train_ids = texts_to_stream(train_texts)
    valid_ids = texts_to_stream(valid_texts)
    print("Train tokens:", len(train_ids), "| Valid tokens:", len(valid_ids))

    # Hiperparámetros (mismo protocolo para GRU y LSTM)
    block_size = 32
    batch_size = 64
    epochs = 5
    lr = 2e-3
    emb_dim = 128
    hidden_dim = 256
    num_layers = 1
    dropout = 0.0
    grad_clip = 1.0

    train_dl = DataLoader(LMDataset(train_ids, block_size), batch_size=batch_size, shuffle=True, drop_last=True)
    valid_dl = DataLoader(LMDataset(valid_ids, block_size), batch_size=batch_size, shuffle=False)

    # 1) Entrenar GRU (base)
    gru_model = train_one_model(
        "gru",
        train_dl, valid_dl,
        vocab_size=len(vocab.itos),
        pad_id=vocab.pad_id,
        device=device,
        emb_dim=emb_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        dropout=dropout,
        lr=lr,
        epochs=epochs,
        grad_clip=grad_clip,
        seed=123
    )

    # Comparación final (valid)
    gru_nll, gru_ppl = evaluate(gru_model, valid_dl, device)
    print("\n=== Comparación final (VALID) ===")
    print(f"GRU : nll={gru_nll:.4f} ppl={gru_ppl:.2f}")

    # Inferencia / generación
    prompt = "the meaning of"
    print("\n--- Generación (GRU) ---")
    print(generate_next_words(gru_model, vocab, prompt, max_new_tokens=25, temperature=0.9, top_k=50, device=device))

    # (Opcional) guardar checkpoints
    torch.save({"model": gru_model.state_dict(), "vocab": vocab}, "gru_lm.pt")
    print("\nGuardado: gru_lm.pt")

In [ ]:
train_and_inference()

# Recurso adicional: Usar RNN para series temporales

En este bloque trabajaremos un problema de predicción de series temporales utilizando una red LSTM. El objetivo es entrenar un modelo que, dado un conjunto de valores pasados de una serie, sea capaz de predecir el siguiente valor. Para ello utilizaremos un dataset público real: Daily Minimum Temperatures in Melbourne (1981–1990), que contiene temperaturas mínimas diarias registradas durante 10 años.

In [ ]:
import math
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# -------------------------
# 1) Cargar dataset público (CSV)
# -------------------------
# Fuente pública (GitHub): daily-min-temperatures.csv
# (Si prefieres no depender de internet en runtime: descarga el CSV y cambia la ruta a local)
CSV_URL = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"

df = pd.read_csv(CSV_URL)
# columnas típicas: Date, Temp
series = df["Temp"].astype("float32").values
print("Total puntos:", len(series))

# -------------------------
# 2) Split train/valid/test (time-based)
# -------------------------
n = len(series)
train_end = int(n * 0.7)
valid_end = int(n * 0.85)

train_raw = series[:train_end]
valid_raw = series[train_end:valid_end]
test_raw  = series[valid_end:]

# Normalización (solo con train)
mu = train_raw.mean()
sigma = train_raw.std() + 1e-8

def norm(x): return (x - mu) / sigma
def denorm(x): return x * sigma + mu

train = norm(train_raw)
valid = norm(valid_raw)
test  = norm(test_raw)

# -------------------------
# 3) Dataset de ventanas
# -------------------------
class WindowDataset(Dataset):
    def __init__(self, arr, window=30):
        self.arr = torch.tensor(arr, dtype=torch.float32)
        self.window = window

    def __len__(self):
        return max(0, len(self.arr) - self.window)

    def __getitem__(self, idx):
        x = self.arr[idx: idx + self.window]          # (T,)
        y = self.arr[idx + self.window]               # ()
        return x.unsqueeze(-1), y.unsqueeze(-1)       # (T,1), (1,)

window = 30
batch_size = 64

train_dl = DataLoader(WindowDataset(train, window), batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(WindowDataset(valid, window), batch_size=batch_size, shuffle=False)
test_dl  = DataLoader(WindowDataset(test, window),  batch_size=batch_size, shuffle=False)

# -------------------------
# 4) Modelo LSTM para regresión 1-step
# -------------------------
class LSTMForecaster(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=1, dropout=0.0):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, h=None):
        # x: (B,T,1)
        out, h = self.lstm(x, h)      # out: (B,T,H)
        last = out[:, -1, :]          # (B,H) último paso temporal
        yhat = self.fc(last)          # (B,1)
        return yhat, h

device = "cuda" if torch.cuda.is_available() else "cpu"
model = LSTMForecaster(hidden_dim=64, num_layers=1).to(device)

# -------------------------
# 5) Train/Eval
# -------------------------
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

@torch.no_grad()
def eval_rmse(dl):
    model.eval()
    se_sum = 0.0
    n = 0
    for x, y in dl:
        x, y = x.to(device), y.to(device)
        yhat, _ = model(x)
        se_sum += ((yhat - y) ** 2).sum().item()
        n += y.numel()
    rmse = math.sqrt(se_sum / max(1, n))
    return rmse

epochs = 10
grad_clip = 1.0

for ep in range(1, epochs + 1):
    model.train()
    total = 0.0
    count = 0
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)
        yhat, _ = model(x)
        loss = loss_fn(yhat, y)

        opt.zero_grad(set_to_none=True)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        opt.step()

        total += loss.item() * y.numel()
        count += y.numel()

    train_mse = total / max(1, count)
    valid_rmse = eval_rmse(valid_dl)
    print(f"epoch {ep:02d} | train_mse {train_mse:.4f} | valid_rmse {valid_rmse:.4f}")

test_rmse = eval_rmse(test_dl)
print("\nTEST RMSE (normalizado):", test_rmse)

# -------------------------
# 6) Inferencia: predecir próximos k pasos (autoregresivo)
# -------------------------
@torch.no_grad()
def forecast_k_steps(model, seed_window, k=7):
    """
    seed_window: array shape (window,) en escala normalizada
    """
    model.eval()
    window_vals = torch.tensor(seed_window, dtype=torch.float32, device=device).view(1, -1, 1)
    preds = []
    h = None
    for _ in range(k):
        yhat, h = model(window_vals, h)          # (1,1)
        next_val = yhat.item()
        preds.append(next_val)
        # slide window: quitar primero, añadir pred al final
        new_seq = torch.cat([window_vals[:, 1:, :], yhat.view(1,1,1)], dim=1)
        window_vals = new_seq
    return preds

# ejemplo con el último window del test
seed = test[:window]
preds_norm = forecast_k_steps(model, seed, k=7)
preds = [denorm(p) for p in preds_norm]
print("\nPredicción próximos 7 días (°C aprox):", [round(p, 2) for p in preds])


Total puntos: 3650
epoch 01 | train_mse 0.7266 | valid_rmse 0.6722
epoch 02 | train_mse 0.4569 | valid_rmse 0.6370
epoch 03 | train_mse 0.4411 | valid_rmse 0.6247
epoch 04 | train_mse 0.4296 | valid_rmse 0.6148
epoch 05 | train_mse 0.4154 | valid_rmse 0.6019
epoch 06 | train_mse 0.3967 | valid_rmse 0.5866
epoch 07 | train_mse 0.3782 | valid_rmse 0.5901
epoch 08 | train_mse 0.3736 | valid_rmse 0.5711
epoch 09 | train_mse 0.3757 | valid_rmse 0.5743
epoch 10 | train_mse 0.3641 | valid_rmse 0.5693

TEST RMSE (normalizado): 0.5493573606485515

Predicción próximos 7 días (°C aprox): [np.float32(7.71), np.float32(7.69), np.float32(7.69), np.float32(7.7), np.float32(7.72), np.float32(7.74), np.float32(7.76)]
